In [13]:
from openai import OpenAI
import httpx
####to investigate
#### decision will be in 2 categories like : code and general, and for general will retrieve from llama3 and the code from codellama
from getpass import getpass
import sys

# https://codellama.130.162.32.162.nip.io/       - helm
# https://llama3.130.162.32.162.nip.io/          - helm
#http://193.122.2.134:8000/v1/                   - first
#http://129.159.199.7.nip.io/v1/                 - first
# agent_llm_api_key = "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"
# code_llm_api_key = "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"


# Update the BaseURL of the model (we are using the service name)
# $ kubectl get service
agent_base_url = "http://agent_llm/v1"
code_base_url = "http://code_llm/v1"

# agent_base_url = "https://nimllama3.130.162.32.162.nip.io/v1/"
# code_base_url = "https://codellama.130.162.32.162.nip.io/v1"


#When using NIM, the llm_api_key is not used, you can provide a dummy value.
agent_llm_api_key  = getpass('Input the agent model API KEY: ')
if not agent_llm_api_key:
    agent_llm_api_key = "dummy"

# When using NIM, the llm_api_key is not used, you can provide a dummy value.
code_llm_api_key  = getpass('Input the code model API KEY: ')
if not code_llm_api_key:
    code_llm_api_key = "dummy"

agent_model_config = {
    "base_url" : agent_base_url,
    "api_key" : agent_llm_api_key,
    "temperature" : 0,
    "top_p" : 1,
    "max_tokens": 256,
    "stream" : True
    }

code_model_config = {
    "base_url" : code_base_url,
    "api_key" : code_llm_api_key,
    "temperature" : 0,
    "top_p" : 1,
    "max_tokens" : 256,
    "stream" : True
    }


httpx_client = httpx.Client(verify=False)
######Modify with client_agent, client_code 
client_agent = OpenAI(
  base_url=agent_model_config["base_url"],
  api_key = agent_model_config["api_key"],
  http_client=httpx_client
)
client_code = OpenAI(
  base_url= code_model_config["base_url"],
  api_key = code_model_config["api_key"],
  http_client=httpx_client
)
def discover_model(client):
#discover available models and use the first one
    available_models = client.models.list()
    if len(available_models.data):
        model = available_models.data[0].id
        print(f"Discovered model is: {model}")
    else:
        print("No model discovered")
        sys.exit(1)
    return model

def create_completion(client, messages, config):
    return client.chat.completions.create(
        model=discover_model(client),
        messages=messages,
        temperature=config["temperature"],
        top_p=config["top_p"],
        max_tokens=config["max_tokens"],
        stream=config["stream"]
    )
def get_decision_from_agent(query):
    messages = [{
        "role": "user",
        "content": f"Please classify the following query into one of the three categories: 'code', 'optimization', or 'general'. 'code' for queries asking for code examples, programming syntax, or specific implementation details. 'optimization' for queries related to improving performance, enhancing efficiency, or refining algorithms. 'general' for queries that do not specifically pertain to coding or optimization but instead involve general concepts, explanations, or broad questions. Answer only with 'code', 'optimization', or 'general': {query}"
    }]

    response = create_completion(client_agent, messages, {**agent_model_config, "max_tokens": 10})
    decision = ""
    for chunk in response:
        if chunk.choices[0].delta.content is not None:
            decision += chunk.choices[0].delta.content
    return decision.strip().lower()

def get_general_response_from_agent(query):
    messages=[{"role":"user","content": query}]
    response = create_completion(client_agent, messages, agent_model_config)
    return response

def get_response_from_code_model(query):
    messages=[{"role":"user","content": query}]
    response = create_completion(client_code, messages, code_model_config)
    codellama_response = ""
    for chunk in response:
      if chunk.choices[0].delta.content is not None:
        codellama_response = codellama_response + chunk.choices[0].delta.content
    print(codellama_response)
    return codellama_response.strip().lower()

def get_preliminary_response(query):
    decision = get_decision_from_agent(query)
    print(f"Decision by Llama3: '{decision}'")
    
    if "code" in decision:
        print("Response generated by: Code Llama")
        response = get_response_from_code_model(query)
    elif "optimization" in decision:
        print("Response generated by: cuopt")
        response = get_response_from_optimization(query)
    else:
        print("Response generated by: Llama3")
        response =  get_general_response_from_agent(query)
    return response

def get_final_response_from_agent_workflow(query, preliminary_response):
    messages=[{"role": "user", "content": f"Use this information {preliminary_response} and generate the response for the question: {query}"}]
    final_response = create_completion(client_agent, messages, agent_model_config)
    return final_response



query = "Write a python code to retrieve the odd numbers in a list"
#query = "Can a python code answer the question how far is the moon ?" #how to make the agent to retrieve the code from the code model ?
preliminary_response = get_preliminary_response(query)
final_response = get_final_response_from_agent_workflow(query, preliminary_response)
print("Final response generated by Llama3")
for chunk in final_response:
  if chunk.choices[0].delta.content is not None:
    print(chunk.choices[0].delta.content, end="")

Input the agent model API KEY:  ········
Input the code model API KEY:  ········


Discovered model is: meta/llama3-8b-instruct
Decision by Llama3: 'code'
Response generated by: Code Llama
Discovered model is: meta-llama/CodeLlama-7b-Instruct-hf
  Here is a Python code to retrieve the odd numbers in a list:
```
def get_odd_numbers(my_list):
    return [x for x in my_list if x % 2 != 0]
```
This code uses a list comprehension to iterate over the elements of the input list `my_list`, and returns a new list containing only the odd numbers. The `if x % 2 != 0` condition checks if the element is odd or not, and the `[x for x in my_list if x % 2 != 0]` syntax creates a new list containing only the elements that satisfy the condition.

You can use this function as follows:
```
my_list = [1, 2, 3, 4, 5, 6, 7, 8, 9]
odd_numbers = get_odd_numbers(my_list)
print(odd_numbers)  # [1, 3, 5, 7, 9]
```
Discovered model is: meta/llama3-8b-instruct
Final response generated by Llama3
Here is the Python code to retrieve the odd numbers in a list:
```
def get_odd_numbers(my_list):
    re